<a href="https://colab.research.google.com/github/husseinhoss155/husseinhoss155/blob/master/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
import numpy as np
import pandas as pd
import _pickle as cPickle
from sklearn.model_selection import train_test_split
import tensorflow
from tensorflow.keras.layers import Conv2D,Activation,Dense,Input,Reshape,Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential

In [56]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [57]:
data = cPickle.load(open('/content/drive/My Drive/Radio dataset/RML2016.10b.dat','rb'),encoding='latin')

In [58]:
series = pd.Series(data)
signals = series.values
lbl = []
for i in series.index:
  for j in range(6000):
    lbl.append([i[0],i[1]])

In [59]:
Xdata=[]
for i in signals:
  for signal in i:
    Xdata.append(signal)
Xdata = np.array(Xdata)

In [60]:
lbl = np.array(lbl)
lbl = lbl[:,0]
lbl = lbl.T
rows = lbl.shape[0]
lbl = lbl.tolist()

In [61]:
np.array(lbl).shape

(1200000,)

In [62]:
dictionary = {'8PSK':0,  'AM-DSB':2,  'AM-SSB':3,'BPSK':4,'CPFSK':5,'GFSK':6,'PAM4':7,'QAM16':8,'QAM64':9,'QPSK':10,'WBFM':1}
for i in range(rows):
  for key,value in dictionary.items():
    if lbl[i] == key:
      lbl[i] = value

In [63]:
# Xdata = Xdata[0:1000,:]
# lbl = np.array(lbl)[0:1000].tolist()

In [64]:
 X_train, X_test, y_train, y_test = train_test_split(Xdata, lbl, test_size=0.3, random_state=1)

In [65]:
 X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.05, random_state=1)

In [66]:
model = Sequential()
model.add(Reshape(target_shape=(2,128,1),input_shape=(2,128)))
model.add(Conv2D(filters=64,kernel_size=(1,3),activation='relu'))
model.add(Conv2D(filters=16,kernel_size=(2,3),activation='relu'))
model.add(Flatten())
model.add(Dense(units=128,activation='relu'))
model.add(Dense(units=11,activation='softmax'))


In [67]:
model.optimizer = tensorflow.keras.optimizers.Adam(learning_rate=0.001)

In [68]:
model.compile(loss=tensorflow.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])

In [69]:
np.array(lbl).shape

(1200000,)

In [70]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_2 (Reshape)         (None, 2, 128, 1)         0         
                                                                 
 conv2d_4 (Conv2D)           (None, 2, 126, 64)        256       
                                                                 
 conv2d_5 (Conv2D)           (None, 1, 124, 16)        6160      
                                                                 
 flatten_2 (Flatten)         (None, 1984)              0         
                                                                 
 dense_4 (Dense)             (None, 128)               254080    
                                                                 
 dense_5 (Dense)             (None, 11)                1419      
                                                                 
Total params: 261,915
Trainable params: 261,915
Non-tr

In [71]:
while True:
  for item in y_train:
    if str(type(item))=='<class \'str\'>':
      print(type(item))
      print(item)
      break
  break

In [ ]:
# X_train = tensorflow.stack(X_train)
# y_train = tensorflow.stack(y_train)
# X_valid = tensorflow.stack(y_train)
# y_valid = 
history = model.fit(
    np.array(X_train),
    np.array(y_train),
    batch_size=32,
    epochs=20,
    validation_data=(np.array(X_valid),np.array(y_valid)),
    shuffle=True
)

Epoch 1/20
 5024/24938 [=====>........................] - ETA: 7:08 - loss: 1.7883 - accuracy: 0.2955